# Hello World of Optimization! 
Let's Make Some Casino Chips
## Our task...
We are in charge of chip production for a casino. 
- Management would like to make these new chips using the on-hand inventory of raw material. 
- Right now, we are asked to manufacture the **highest possible total value** in poker chips. 
- The denominations we can make are \\$1, \\$5, \\$10, \\$25, \\$100, \\$500, \$1000.
- Each denomination of chip requires a different amount of several raw materials.
- We are limited to the amount of material **on hand**.

## Initial questions
- What are our decisions?
- What are our constraints? 
- What is the objective?
- What data do we need right now?

## The Model
A lot of programming (in Python) is imperative -- just providing sequential instructions to complete. But mathematical optimization (aka math programming) is *declarative*. The math programming model does not tell the Gurobi solver what to do specifically. Instead, the model tells the Gurobi solver what the solution must look like. Gurobi then finds the solution in its own way.

So math programming always starts with the creation of a new model. We then add to it the *declarations* about the final solution. 

Math optimization models take two forms:
- The **formulation**: An algebraic representation of the model.
- The **code**: Writing the formulation in syntax to some software package.

In [4]:
import importlib.util
import subprocess
import sys

if importlib.util.find_spec('gurobipy') is None:
    print(f"Package '{'gurobipy'}' is not installed. Installing now...")
    # Install the package using pip
    subprocess.check_call([sys.executable, "-m", "pip", "install", 'gurobipy'])
else:
    print(f"Package '{'gurobipy'}' is already installed.")

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

Package 'gurobipy' is already installed.


In [5]:
model = gp.Model("Chip Manufacturing")

### The Variables

The model needs something to receive the solution values that the solver finds. It declares *variables* to hold those solution values.  The mathematical optimization model never explicitly sets these values. It just describes them and makes rules about the values they can hold. This model declares a variable for each type of chip.

Let $x_c$ be the number of chips made of type $c \in \{\$1, \$5, \$10, \$25, \$100, \$500, \$1000\}$.

First, let's create data for each chip type and its value.

In [6]:
chips = ["one", "five", "ten", "twenty-five", "one hundred", "five hundred", "thousand"]
data = pd.Series([1, 5, 10, 25, 100, 500, 1000], 
                  index=chips, 
                  name='value')
data

one                1
five               5
ten               10
twenty-five       25
one hundred      100
five hundred     500
thousand        1000
Name: value, dtype: int64

#### Anyone know another way to hardcode this another way?
Using a [multidict](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/func_global.html#multidict) object is a fast way to hardcode data in `gurobipy`.

In [7]:
chips, value = gp.multidict(
    {
        "one":             1,
        "five":            5,
        "ten":            10,
        "twenty-five":    25,
        "one hundred":   100,
        "five hundred":  500,
        "thousand":     1000
    }
)
print(chips)
print(value)

['one', 'five', 'ten', 'twenty-five', 'one hundred', 'five hundred', 'thousand']
{'one': 1, 'five': 5, 'ten': 10, 'twenty-five': 25, 'one hundred': 100, 'five hundred': 500, 'thousand': 1000}


Next, variables are added to the model -- one for each chip type. The main types of decision variables are
- CONTINUOUS
- INTEGER
- BINARY

But we'll explore more types later. 

#### What type of decision variable should we use in this case?

In [8]:
x = model.addVars(chips, vtype = GRB.INTEGER, name = "chips")
model.update()
x

{'one': <gurobi.Var chips[one]>,
 'five': <gurobi.Var chips[five]>,
 'ten': <gurobi.Var chips[ten]>,
 'twenty-five': <gurobi.Var chips[twenty-five]>,
 'one hundred': <gurobi.Var chips[one hundred]>,
 'five hundred': <gurobi.Var chips[five hundred]>,
 'thousand': <gurobi.Var chips[thousand]>}

### The Objective Function:

The math model must describe an objective also using algebra. When the model is solved, the value of the objective function will be the maximum or minimum possible while following the rules described in the in constraints. 

This model will produce the highest possible value of chips. The objective function multiplies the quantity of each chip produced times its value and that for all chips.
For example, the total value produced by \\$5 chips is $5*x_{five}$. 

So the total value is

\begin{equation*}
x_{one} + 5*x_{five} + 10*x_{ten} + ... + 1000*x_{thousand}
\end{equation*}

In [9]:
### Written term by term:
model.setObjective(x["one"] * value["one"] + x["five"] * value["five"] + x["ten"] * value["ten"]
                    + x["twenty-five"] * value["twenty-five"] + x["one hundred"] * value["one hundred"] 
                    + x["five hundred"] * value["five hundred"] + x["thousand"] * value["thousand"], 
                    sense=GRB.MAXIMIZE)

Let $v_c$ be the value of chip $c$ and use a bit more math notation:
\begin{equation*}
  \text{Maximize} \space \sum_c v_c*x_c
\end{equation*}

In [10]:
### Here is another way to write the same thing:
model.setObjective(gp.quicksum(x[i] * value[i] for i in chips), sense=GRB.MAXIMIZE)

### The prod function is a handy function provided by the Gurobi API. 
model.setObjective(x.prod(value), sense=GRB.MAXIMIZE)

### The Constraints:

This casino has limited ingredients on hand to manufacture the chips needed. The model must make sure to only make chips for which the ingredients are available. 

Here is a list of ingredients and how much is needed for each type of chip:

In [11]:
ingredients, on_hand = gp.multidict(
    {
        "clay":      5000,
        "lead":      1800,
        "silver":     100,
        "gold":        20,
    }
)

recipes = { 
        ("one",          "clay"):  18.0, ("one",          "lead"):  0.0, ("one",          "silver"):  0, ("one",          "gold"):  0, 
        ("five",         "clay"):  17.0, ("five",         "lead"):  2.0, ("five",         "silver"):  0, ("five",         "gold"):  0, 
        ("ten",          "clay"):  16.0, ("ten",          "lead"):  3.0, ("ten",          "silver"):  0, ("ten",          "gold"):  0, 
        ("twenty-five",  "clay"):  13.0, ("twenty-five",  "lead"):  4.5, ("twenty-five",  "silver"):  0, ("twenty-five",  "gold"):  0, 
        ("one hundred",  "clay"):  10.0, ("one hundred",  "lead"):  6.0, ("one hundred",  "silver"):  1, ("one hundred",  "gold"):  0, 
        ("five hundred", "clay"):  10.0, ("five hundred", "lead"):  8.5, ("five hundred", "silver"):  2, ("five hundred", "gold"):  0, 
        ("thousand",     "clay"):  10.0, ("thousand",     "lead"):  9.5, ("thousand",     "silver"):  0, ("thousand",     "gold"):  2, 
}
print(ingredients)


['clay', 'lead', 'silver', 'gold']


In [12]:
on_hand

{'clay': 5000, 'lead': 1800, 'silver': 100, 'gold': 20}

Let's introduce **constraints** to make sure we don't use more ingredients than we have on hand. Constraints are where the *rules* acting on decision variables are declared. 

For example, the amount of lead used for all chips made must be less than or equal to the total amount of lead on hand. 

\begin{equation*}
\text{total lead used} \le 1500
\end{equation*}

Let's write an expression for the total lead used using our decision variable for lead, $x_{lead}$.
\begin{align*}
\text{total lead used} = \space& lead_{one}* x_{one} + lead_{five}*x_{five} + ... + lead_{thousand}*x_{thousand} \\
&lead_{one}* x_{one} +  lead_{five}*x_{five} + ... + lead_{thousand}*x_{thousand} \le 1500
\end{align*}

Given we stored this information in the `recipes` dictionary, we let $r_{c,i}$ be the amount of *ingredient* $i$ used in making *chip* $c$.
\begin{equation*}
  r_{one, lead} * x_{one} + r_{five, lead}*x_{five} + ... + r_{thousand, lead}*x_{thousand} \le 1500
\end{equation*}


In [13]:
# A very explicit way to write these constraints:
model.addConstr(x["one"]  * recipes["one", "clay"] + x["five"]  * recipes["five", "clay"] 
                + x["ten"]  * recipes["ten", "clay"] + x["twenty-five"]  * recipes["twenty-five", "clay"] 
                + x["one hundred"]  * recipes["one hundred", "clay"] + x["five hundred"]  * recipes["five hundred", "clay"] 
                + x["thousand"] * recipes["thousand", "clay"] <= on_hand["clay"], "clay limit")

model.addConstr(x["one"]  * recipes["one", "lead"] + x["five"]  * recipes["five", "lead"] 
                + x["ten"]  * recipes["ten", "lead"] + x["twenty-five"]  * recipes["twenty-five", "lead"] 
                + x["one hundred"]  * recipes["one hundred", "lead"] + x["five hundred"]  * recipes["five hundred", "lead"] 
                + x["thousand"] * recipes["thousand", "lead"] <= on_hand["lead"], "lead limit")

model.addConstr(x["one"]  * recipes["one", "gold"] + x["five"]  * recipes["five", "gold"] 
                + x["ten"]  * recipes["ten", "gold"] + x["twenty-five"]  * recipes["twenty-five", "gold"] 
                + x["one hundred"]  * recipes["one hundred", "gold"] + x["five hundred"]  * recipes["five hundred", "gold"] 
                + x["thousand"] * recipes["thousand", "gold"] <= on_hand["gold"], "gold limit")

model.addConstr(x["one"]  * recipes["one", "silver"] + x["five"]  * recipes["five", "silver"] 
                + x["ten"]  * recipes["ten", "silver"] + x["twenty-five"]  * recipes["twenty-five", "silver"] 
                + x["one hundred"]  * recipes["one hundred", "silver"] + x["five hundred"]  * recipes["five hundred", "silver"] 
                + x["thousand"] * recipes["thousand", "silver"] <= on_hand["silver"], "silver limit")

<gurobi.Constr *Awaiting Model Update*>

We can also generalize the *quantity* of each ingredient with $q_i$. Then using a bit more mathematical notation:
\begin{equation*}
  \sum_{c}r_{c, i} * x_{c} \le q_i, \space \text{for all} \space i \space \text{in} \space \{\text{clay, lead, silver, gold}\}
\end{equation*}

Note: A short way to write "for all" is using the symbol $\forall$. Also, $\in$ means "in", or "an element of."

We can change the code so it looks for like this condensed notation and loop through each ingredient using `quicksum`: 

In [14]:
for ingredient in ingredients:
    model.addConstr(gp.quicksum(x[c] * recipes[c, ingredient] for c in chips) <= on_hand[ingredient], name="ingredients usage")

A little more compact, and makes it easier to store the constraints as an object:

In [15]:
balance_constraints = model.addConstrs((gp.quicksum(x[c] * recipes[c, i] for c in chips) <= on_hand[i] for i in ingredients), name="ingredients usage")

#### Note!
If you get the following error...
```python
SyntaxError: Generator expression must be parenthesized
```
...make sure you have everything before `, name=` in parentheses. 

### The Solution:
It's as simple as one line of code to run the optimization, then we query the decision variables for their values (assuming the optimization completed successfully)

In [16]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.6.0 23G80)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 7 columns and 48 nonzeros
Model fingerprint: 0x15a486a4
Variable types: 0 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+03]
Found heuristic solution: objective 302.0000000
Presolve removed 9 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 7 columns, 15 nonzeros
Variable types: 0 continuous, 7 integer (0 binary)
Found heuristic solution: objective 1876.0000000

Root relaxation: objective 4.215012e+04, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 42150.1235    

In [17]:
# use `VarName` and `X` to get the variables name and value, respectively:
for v in model.getVars():
    print(f"{v.VarName}: {v.X}")

chips[one]: 38.0
chips[five]: 1.0
chips[ten]: 0.0
chips[twenty-five]: 284.0
chips[one hundred]: 0.0
chips[five hundred]: 50.0
chips[thousand]: 10.0


Do you think another (and simpler) approach may work for this specific version of the problem?

## Congratulations!
You have just ran an optimization model!